In [14]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
print(os.listdir("../input"))

['train.csv', 'sample_submission.csv', 'test.csv']


In [15]:
def plot_eval_result(clf, figsize=(20,8), title='Evaluation Result'):
    eval_result = clf.evals_result()
    plt.figure(figsize=figsize)
    plt.plot(eval_result['validation_0'][metric])
    plt.plot(eval_result['validation_1'][metric])
    plt.xlabel('Epoch')
    plt.ylabel(metric)
    plt.show()

In [16]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [17]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

x_train, x_valid, y_train, y_valid = train_test_split(
    train.iloc[:, 1:],
    train.iloc[:, 0],
    stratify = train.iloc[:, 0],
    test_size = 0.2,
    random_state = 1998, 
    shuffle = True
)
x_train, x_valid, x_test = x_train/255, x_valid/255, test/255
y_train, y_valid = to_categorical(y_train, 10), to_categorical(y_valid, 10)

In [21]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.backend import set_session
from keras.optimizers import SGD
import tensorflow as tf

# Set GPU Availability:
config = tf.ConfigProto( device_count = {'GPU': 1, 'CPU': 100} ) 
sess = tf.Session(config=config) 
set_session(sess)

clf = Sequential([
    Dense(784, input_shape = (784,), activation='relu'),
    Dense(1024, activation='relu'),
    Dense(512, activation='relu'),
    Dense(10, activation='softmax')
])

clf.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.1),  metrics=['accuracy'])

clf.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=20)

Train on 33600 samples, validate on 8400 samples
Epoch 1/20
33600/33600 [==============================] - 6s 190us/step - loss: 0.3073 - acc: 0.9058 - val_loss: 0.2078 - val_acc: 0.9333
Epoch 2/20
33600/33600 [==============================] - 5s 161us/step - loss: 0.1169 - acc: 0.9634 - val_loss: 0.0990 - val_acc: 0.9690
Epoch 3/20
33600/33600 [==============================] - 5s 160us/step - loss: 0.0735 - acc: 0.9768 - val_loss: 0.0951 - val_acc: 0.9705
Epoch 4/20
33600/33600 [==============================] - 5s 160us/step - loss: 0.0472 - acc: 0.9853 - val_loss: 0.0828 - val_acc: 0.9745
Epoch 5/20
33600/33600 [==============================] - 5s 161us/step - loss: 0.0312 - acc: 0.9904 - val_loss: 0.0914 - val_acc: 0.9732
Epoch 6/20
33600/33600 [==============================] - 5s 160us/step - loss: 0.0224 - acc: 0.9929 - val_loss: 0.1151 - val_acc: 0.9686
Epoch 7/20
33600/33600 [==============================] - 5s 161us/step - loss: 0.0158 - acc: 0.9950 - val_loss: 0.0754 - v

In [19]:
y_pred = pd.DataFrame(data = clf.predict_classes(x_test), index=list(range(1, test.shape[0]+1) ), columns = ['Label'])
y_pred.to_csv("output.csv", index=True, index_label='ImageId')